In [63]:
import os
import re
import json
import requests
from datetime import datetime
from typing import Any
from collections import defaultdict

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://nl.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikipedia:Wist_je_dat'

OUTPUT_DIR = 'data/nl'


def get_day_links_from_archive(main_page_url: str) -> dict[str, dict[str, Any]]:
    """
    Get the year links from the main archive page.
    Current year link is the main page.
    """
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    div = soup.find('div', class_='mw-heading mw-heading2 ext-discussiontools-init-section')
    if not div:
        raise ValueError("Could not find the div.")
    
    table = div.find_next('table', class_='wikitable')
    if not table:
        raise ValueError("Could not find the table.")

    archive_links = {}
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')

        month = tds[0].find('a', href=True).text.strip()

        for a_tag in tds[1].find_all('a', href=True):        
            day = a_tag.text.strip()
            href = a_tag['href']
            full_url = urljoin(BASE_URL, href)
            exists = 'new' not in a_tag.get('class', [])

            full_day_name = day + ' ' + month
            archive_links[full_day_name] = {
                'url': full_url,
                'exists': exists
            }

    return archive_links


In [64]:
archive = get_day_links_from_archive(MAIN_URL)

archive

{'1 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_1_januari',
  'exists': True},
 '2 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_2_januari',
  'exists': True},
 '3 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_3_januari',
  'exists': True},
 '4 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_4_januari',
  'exists': True},
 '5 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_5_januari',
  'exists': True},
 '6 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_6_januari',
  'exists': True},
 '7 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_7_januari',
  'exists': True},
 '8 Januari': {'url': 'https://nl.wikipedia.org/wiki/Sjabloon:Hoofdpagina_-_wist_je_dat_8_januari',
  'exists': True},
 '9 Januari': {'url': 'https://nl.wikipedia.org/

In [65]:
def _extract_fact_data(element: Tag, base_url: str) -> dict[str, Any]:
    """Extracts text, links, and relevant links from a BeautifulSoup Tag."""
    fact_text = "Wist je dat " + element.get_text(" ", strip=True).lstrip().lstrip('.').lstrip('…').lstrip()
    fact_text = fact_text.replace('  ', ' ')
    fact_text = fact_text.replace('\xa0', ' ')

    links = []
    for a in element.find_all("a", href=True):
        href = a["href"]
        if not href.startswith("/wiki/"):
            continue

        full_url = unquote(urljoin(base_url, href))

        links.append(full_url)

    return {
        "text": fact_text,
        "links": links,
        "relevant_links": [],
    }


def parse_day_facts(day_url: str, day: str) -> list[dict]:
    """Parse the facts from the day page."""
    resp = requests.get(day_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    year_div = soup.find("div", class_="documentatie")
    year = None
    if year_div:
        year_p = year_div.find("p")
        if year_p:
            year_text = year_p.get_text(" ", strip=True)
            match = re.search(r'\d{4}', year_text)
            if match:
                year = match.group(0)

    results: list[dict] = []

    div = soup.find("div", class_="mw-content-ltr mw-parser-output")
    if not div:
        return []

    ul = div.find("ul")
    if not ul:
        return []

    for li in ul.find_all("li"):
        fact_data = _extract_fact_data(li, day_url)

        if fact_data['text']:
            results.append({
                "section": day + ' ' + year,
                **fact_data
            })

    return results

In [68]:
day = '13 Februari'

facts = parse_day_facts(archive[day]['url'], day)
facts

[{'section': '13 Februari 2015',
  'text': 'Wist je dat de emir van Abu Dhabi tevens president is van de Verenigde Arabische Emiraten ?',
  'links': ['https://nl.wikipedia.org/wiki/Abu_Dhabi_(emiraat)',
   'https://nl.wikipedia.org/wiki/Verenigde_Arabische_Emiraten'],
  'relevant_links': []},
 {'section': '13 Februari 2015',
  'text': 'Wist je dat in 1947 de eerste film over De avonturen van Kuifje werd uitgebracht?',
  'links': ['https://nl.wikipedia.org/wiki/De_krab_met_de_gulden_scharen_(film)',
   'https://nl.wikipedia.org/wiki/De_avonturen_van_Kuifje'],
  'relevant_links': []},
 {'section': '13 Februari 2015',
  'text': 'Wist je dat Thiomargarita namibiensis de grootste bacterie ter wereld is, met afmetingen tot driekwart millimeter?',
  'links': ['https://nl.wikipedia.org/wiki/Thiomargarita_namibiensis'],
  'relevant_links': []},
 {'section': '13 Februari 2015',
  'text': 'Wist je dat Huub Stapel en Diederik Stapel (voor zover bekend) geen familie van elkaar zijn, alhoewel beiden

In [53]:
MONTHS = {
    'січень': 'січень', 'січня': 'січень',
    'лютий': 'лютий', 'лютого': 'лютий',
    'березень': 'березень', 'березня': 'березень',
    'квітень': 'квітень', 'квітня': 'квітень',
    'травень': 'травень', 'травня': 'травень',
    'червень': 'червень', 'червня': 'червень',
    'липень': 'липень', 'липня': 'липень',
    'серпень': 'серпень', 'серпня': 'серпень',
    'вересень': 'вересень', 'вересня': 'вересень',
    'жовтень': 'жовтень', 'жовтня': 'жовтень',
    'листопад': 'листопад', 'листопада': 'листопад',
    'грудень': 'грудень', 'грудня': 'грудень',
}

def extract_month_from_title(title: str) -> str | None:
    """Extracts the first mentioned month from a section title."""
    words = re.split(r'[\s—-]+', title)

    for word in words:
        cleaned_word = word.lower().strip()
        
        if cleaned_word in MONTHS:
            return MONTHS[cleaned_word]

    return None

In [60]:
extract_month_from_title('3 22 грудня 2022 — 23 січня 2023')

'грудень'

In [52]:
import dateparser

year = '2025'
month = '7'
section = '11 — 25 лютого 2025'

dt_month = dateparser.parse(section)
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-02'